# Importing Libraries 

In [4]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [6]:

import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os



In [7]:
dataset = '/Users/Ivan_Feng_Jun_Kai/Desktop/SUTD/UROP-AI with Cancer Dianogsis/Video Analysis/video_classification/street_dataset/data'
trained_model = '/Users/Ivan_Feng_Jun_Kai/Desktop/SUTD/UROP-AI with Cancer Dianogsis/Video Analysis/video_classification/model'
label_bin = '/Users/Ivan_Feng_Jun_Kai/Desktop/SUTD/UROP-AI with Cancer Dianogsis/Video Analysis/video_classification/model/lb.pickle'
epochs = 1
plot = 'output/plot.png'
# initialize the set of labels from the spots activity dataset we are
# going to train our network on
LABELS = set(["badminton", "basketball", "fencing"])



In [8]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images(dataset))
data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]

    # if the label of the current image is not part of of the labels
    # are interested in, then ignore the image
    if label not in LABELS:
        continue

    # load the image, convert it to RGB channel ordering, and resize
    # it to be a fixed 224x224 pixels, ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))

    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)


[INFO] loading images...


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [9]:
# convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, stratify=labels, random_state=42)

In [10]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [11]:
pip install 'h5py<3.0.0'

Note: you may need to restart the kernel to use updated packages.


In [12]:
baseModel = ResNet50(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

2022-06-17 17:27:35.009162: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-06-17 17:27:35.027974: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fbd394b8c00 executing computations on platform Host. Devices:
2022-06-17 17:27:35.027993: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version


In [13]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
    layer.trainable = False

In [14]:
epochs = 5

# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / epochs)
model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])

# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit_generator(
    trainAug.flow(trainX, trainY, batch_size=25),
    steps_per_epoch=len(trainX) // 32,
    validation_data=valAug.flow(testX, testY),
    validation_steps=len(testX) // 32,
    epochs=epochs)

[INFO] compiling model...
[INFO] training head...
Epoch 1/5
47/47 [==============================] - 567s 12s/step - loss: 1.3510 - accuracy: 0.3813 - val_loss: 1.0281 - val_accuracy: 0.5250
Epoch 2/5
47/47 [==============================] - 579s 12s/step - loss: 1.0396 - accuracy: 0.5259 - val_loss: 0.8449 - val_accuracy: 0.6104
Epoch 3/5
47/47 [==============================] - 527s 11s/step - loss: 0.8811 - accuracy: 0.5995 - val_loss: 0.7404 - val_accuracy: 0.6500
Epoch 4/5
47/47 [==============================] - 527s 11s/step - loss: 0.7643 - accuracy: 0.6706 - val_loss: 0.6810 - val_accuracy: 0.6896
Epoch 5/5
47/47 [==============================] - 521s 11s/step - loss: 0.6614 - accuracy: 0.7137 - val_loss: 0.6470 - val_accuracy: 0.7042


In [17]:
# plot the training loss and accuracy
N = epochs
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(plot)

In [18]:


# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX.astype("float32"), batch_size=32)
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1), target_names=lb.classes_))



[INFO] evaluating network...
              precision    recall  f1-score   support

   badminton       0.61      0.98      0.75       232
  basketball       0.94      0.47      0.63       123
     fencing       0.97      0.48      0.64       156

    accuracy                           0.71       511
   macro avg       0.84      0.65      0.68       511
weighted avg       0.80      0.71      0.69       511



In [20]:
# serialize the model to disk
trained_model = '/Users/Ivan_Feng_Jun_Kai/Desktop/SUTD/UROP-AI with Cancer Dianogsis/Video Analysis/video_classification/model/activity.model'
print("[INFO] serializing network...")
model.save(trained_model, save_format="h5")

[INFO] serializing network...


In [21]:
# serialize the label binarizer to disk
f = open(label_bin, "wb")
f.write(pickle.dumps(lb))
f.close()